# Data Science Society Term 2 Workshop 1 - Intro to Deep Learning
Welcome back to our series of Python tutorials! Hopefully you are all suitably refreshed from the holidays. Last term we introduced the basics of Python and in particular explored the Pandas and Sci-Kit Learn libraries to start building a few machine learning models. This term we will look at more complicated models focusing in particular on neural networks and how we can use these for classification. In the folder this notebook is in you will find a basic introduction into neural networks that should hopefully demystify some of what we do today. Today we will look at a dataset which consists of images of various characters from $0-9$. We will run through how to construct a model consisting of neural network that will predict the character in the image. We will also call upon the aid of the Keras library in order to build our neural networks. As you may not have Keras installed on your system relevant shell commands for installation are provided in the two code cells below. Run these before proceeding.

In [ ]:
!pip install keras

In [ ]:
!pip install theano

### Importing Libraries 
As is convention, we begin by importing all the relavent libraries for todays workshop.

In [ ]:
import os
os.environ['KERAS_BACKEND'] = "theano"
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Aqcuiring the Data
Usually preprocessing can be something of a task in and of itself for machine learning problems of this nature. However, here we use some preprepared data already in a form thats ready to use. We will cover how to load your own data in a future workshop. For now all the training and test data are conventiently arranged for us. All the code below does is it loads the ````mnist```` dataset from the ````datasets```` module of keras. 

In [ ]:
# Loads mnist data set from keras.datasets
mnist = keras.datasets.mnist

# Define our training and testing data variables from the mnist dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

### What is the Data About?
To begin, its always nice to have some context so in that spirit lets take a look at some of the data. First lets print ````X_train```` to see what we get.

In [ ]:
print(X_train)

What we see above is a lot of arrays. But if you look closely at the brackets you'll see arrays within arrays. Indeed you might be familair with $2D$ arrays which are often referred to as matrices. What we have here is a larger array that seemingly stores some number of these $2D$ arrays. We can see this if we ask for the shape of ````X_train````.

In [ ]:
# Returns the shape of X_train
X_train.shape

As we can see the output returns three numbers the last two of which refer to the dimensions of the matrices and the first one merely refers to which matrix your using. These $3D$ arrays are often referred to as "tensors" hence the name tensor flow is used for these kinds of problems. I use quotation marks because simply having this structure is not enough to qualify as a tensor but enough on that. Essentially, all we are doing is using this "tensor" as a store of smaller arrays. Each of the $2D$ arrays represents one case from our dataset. Let us display one of these arrays to see what it represents. Below we invoke ````plt.imshow```` in order to display the first matrix from our "tensor".

In [ ]:
# Displays first matrix from X_train "tensor" 
plt.imshow(X_train[0])
# Includes scale
plt.colorbar()

As we can see above the values in the matrix above have been plotted into an image that we can see. In this case we have the number $5$. We want to be able to build a model that is able to distinguish between all the different numbers from $0-9$. This is where we deploy the neural network with the aid of Keras. Before this however we note from the colour bar the actual values in the matrices range from $0-250$. 
## Normalisation
When feeding this data to our neural network ideally this data is normalised meaning all values range from $0-1$. Luckily for us Keras has means a means of normalising our data so we will just use the Keras functions for this purpose. Here we use ````keras.utils.normalize```` with the first argument being the data we wish to normalise and the second argument ````axis=1```` specifies that we do so down the matrix columns.

In [ ]:
# Normalising the data with respect to the column of each matrix
X_train = keras.utils.normalize(X_train, axis=1)
X_test = keras.utils.normalize(X_test, axis=1)

Below we display our the first matrix again to demonstrate what has changed. As we can see the scale is now reduced to be from $0$ to $1$.

In [ ]:
# Displays first matrix from X_train "tensor" 
plt.imshow(X_train[0])
# Includes scale
plt.colorbar()

## Building the Neural Network
Now to begin building the neural network! We start by defining our model as ````keras.models.Sequential()```` where the ````Sequential```` model is a neural network with a linear stack of layers. This step merely decides what type of neural network we are using. None of the structure has been defined. We will have to do that ourselves. To devolop our neural network we pass arguments into ````model.add````.

First we add ````Flatten()```` meaning all the numbers in each matrix are passed individually rather than the matrix as a whole. This is our input layer.

Next we need to consider our hidden layers. To add a layer we use pass into ````model.add```` the argument ````Dense(32, activation="relu")````. Here ````Dense```` merely refers to the fact we are adding a layer. The first argument in ````Dense```` specifies the number of neurons there are to be in that layer. In this case we've chosen ````32````. The ````activation="relu"```` argument determines the activation function that is to be used. Here ````relu```` refers to a rectified linear activation function. We add two such hidden layers before proceeding.

The output layer is created in a similar fashion albiet with a few different arguments. Namely, we need ````10```` neurons for this layer because that is how many possible outcomes there are in are in our dataset (ie each image is any number from $0-9$). We also change to a ````softmax```` activation function for this layer as gives us a probability distribution as output. 

One should note that the code below merely defines the structure our neural network. No training has yet occured. Additionaly, those interested in the specifics behind the activation functions then some mathmatical detail has been provided at the end of this notebook if you are interested. The rectified linear function is rather straightforward wheras the softmax is less so.

In [ ]:
# Defining our model as a linear stack of layers
model = Sequential()

# Flattens the data before passing into our neural network (input layer)
model.add(Flatten())

# Creating our two hidden layers each with 32 neurons and the rectified linear activation function
model.add(Dense(32, activation="relu"))
model.add(Dense(32, activation="relu"))

# Creating the output layer with 10 neurons and the softmax activation function
model.add((Dense(10, activation="softmax")))

## Model Training
Before training we use ````model.compile```` to specify the ways in which the training will occur. First, we choose a type of ````optimizer````. There are many different choices one could choose from here but in essence these simply revolve around different methods of finding minima or maxima. The ````adam```` optimizer is a popular default choice. Next we choose a method for determining ````loss````. Loss is a measure of the error and therefore something we seek to minimise. For this we have choosen ````sparse_categorical_crossentropy````. We choose also choose which ````metrics```` to track by passing a list for this argument. Here we choose to track ````accuracy```` as this is not tracked by default.

Now all that is left is to train the model. We do this by passing ````model.fit(X_train,y_train, epochs=3)```` where here ````epochs=3```` merely refers to the number of times we wish to run the training.

In [ ]:
# Defines how the model will be trained
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

# Trains the model with three runs
model.fit(X_train,y_train, epochs=3)

## Model Evaluation
Below we use ````model.evaluate```` to record the loss and accuracy before printing these two parameters out.

In [ ]:
# Returns the values of loss and accuracy
loss, acc = model.evaluate(X_test, y_test)
print(loss, acc)

As we can see above we should have what looks to be a fairly decent score. However, its all well and good to see such scores but it feels far more tangable when we have a solid prediction we can see so lets test it out. Below we'll give our model the first matrix from the ````X_test```` set. This is not quite as straightforward as the ````predict```` method returns the probability distrubtions. To rectify this we call upon ````argmax```` function from Numpy to give us a result that is more familair to us.

In [ ]:
# Converts test matrices into probability distributions 
predictions = model.predict(X_test)

# Prediction from first test matrix
print(np.argmax(predictions[0]))

Let us now plot the first matrix to see if it matches our prediction.

In [ ]:
# Displays first matrix from X_train "tensor" 
plt.imshow(X_test[0])
# Includes scale
plt.colorbar()

Our model successfully predicted a $7$!

## Saving and Loading a Model
One can save existing models and then load them later to avoid the need for repeated training. Saving is done simply through ````model.save("filename.model")```` and loading through ````keras.models.load_model("filename.model")````.

In [ ]:
# Saving a model as "num_reader.model"
model.save("num_reader.model")

In [ ]:
# Loading the saved model from the "num_reader.model" file
model = keras.models.load_model("num_reader.model")

## More on Activation Functions (Optional)
### Rectified Linear
This was the activation function we used for our hidden layers. Simply put the function is defined:
$$ y = \begin{cases} 0 & for \>\>\>\> x < 0 \\ x & for \>\>\>\> x > 0\end{cases}. $$ Essentially any value of $x$ less than $0$ returns $0$ while any value of $x$ greater than $0$ returns $x$. It's simplicity means it has a low computational cost associated with it making it a popular choice as an activation function. Below I've generated a plot of the activation function below.

In [ ]:
x = np.linspace(-1,1,1000)
y = np.zeros(1000)
for i in range(len(x)):
    if x[i] > 0:
        y[i] = x[i]

plt.grid()
plt.plot(x,y)

### Softmax
The softmax activation function defines the probability that a system is in state $z_i$ as follows: $$ P(z_i) = \frac{e^{-\beta z_i}}{\sum_j e^{-\beta z_j}}. $$ Here the summation is over all possible values of $j$ and the factor on the denominator serves to normalise the probability distribution. The equation is also referred to as the Boltzmann distribution in statistical mechanics. In the context of thermodynamics $z_i$ would represent the energy of individual microstates and $\beta$ would be the inverse temperature. The denominator is also referred to as the partition function.